In [1]:
import warnings

import numpy as np
import pandas as pd

from gensim.models.doc2vec import Doc2Vec
from surprise import Dataset, Reader
from tqdm import tqdm

from src.models import cf, evaluate_model
from src.utilities import utilities

tqdm.pandas()
warnings.filterwarnings('ignore')

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data and Models

In [2]:
# global variables
DATA_PATH = "data/evaluation"
D2V_PATH = "models/d2v"
CATEGORY = "Pet_Supplies"

# d2v training parameters
EPOCHS = 10

# training parameters
N_EPOCHS = 15
LR_ALL = 0.01
BETA = 0.1

# reproducibility checks
SEED = 42
np.random.seed(42)

train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")
d2v = Doc2Vec.load(f"{D2V_PATH}/{CATEGORY}_user_item_50_{EPOCHS}_d2v.model")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A14CK12J7C7JRK,3.0,I purchased the Trilogy with hoping my two cat...,2011-01-12,purchase trilogy hop cat age interested yr old...
1,2,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2CR37UY3VR7BN,4.0,I bought the triliogy and have tested out all ...,2012-12-19,buy triliogy test dvd appear volume receive re...
2,3,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2A4COGL9VW2HY,4.0,My female kitty could care less about these vi...,2011-05-12,female kitty care video care little male dig a...
3,4,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2UBQA85NIGLHA,3.0,"If I had gotten just volume two, I would have ...",2012-03-05,volume star trilogy star read review know vol ...
4,5,B00005MF9U,LitterMaid LM900 Mega Self-Cleaning Litter Box,"['Pet Supplies', 'Cats', 'Litter &amp; Housebr...",A2BH04B9G9LOYA,1.0,"First off, it seems that someone is spamming t...",2006-12-31,spamming review glow reviewer review amazon ba...
68865,111581,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A11J1FHCK5U06J,4.0,Now I know exactly where the trouble spots are...,2014-05-23,know exactly trouble spot sniffing guess invis...
68866,111585,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A18JF0T0GOCORW,4.0,I use this light to help me find stains when I...,2014-05-24,use light help stain carpet clean pre treat ca...
68867,111595,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A3GRPCW9DG427Z,5.0,We are owned by the 3 pickiest pooches in the ...,2013-07-27,pickiest pooch world love fool reject doggie t...
68868,111598,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A2X6TLAX3JEO1A,5.0,My highly allergic white boxer loves these tre...,2014-05-09,highly allergic white boxer love treat meat co...
68869,111602,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A9PG9ODPPP31N,5.0,Works great on my medium sized dog. She has ve...,2014-07-09,work great medium size dog coarse hair work gr...


In [4]:
# testing d2v models
d2v.dv[0]

array([-0.2520061 , -0.12801759, -0.01807678, -0.06055114, -0.04010666,
        0.04099023,  0.0625717 ,  0.00891204, -0.13964826,  0.08402346,
        0.0650855 ,  0.13047956,  0.03227995,  0.03988666, -0.07124443,
       -0.06424009,  0.02801364, -0.00741817, -0.09139743,  0.02618102,
       -0.01038758,  0.00207332, -0.13542707, -0.09922392, -0.02690951,
        0.11702102,  0.03746996, -0.1585136 , -0.08835025, -0.13124834,
        0.02231358,  0.05603463, -0.0014683 ,  0.0878399 ,  0.04198717,
        0.19678356, -0.13217868, -0.15097322,  0.16004062, -0.01319988,
       -0.02964439,  0.06364242, -0.0199494 ,  0.00027959,  0.01765113,
        0.14233795,  0.11001149, -0.06930998, -0.03381407, -0.05987406],
      dtype=float32)

# Generating User & Item Embeddings

In [5]:
user_idx_map, user_vecs, item_idx_map, item_vecs = utilities.generate_user_item_embeddings(train, d2v)

In [6]:
# converting factors into numpy obj
user_factors = user_vecs.to_numpy()
item_factors = item_vecs.to_numpy()

In [7]:
# check user factors
user_factors[0,:]

array([-0.8561372 , -0.08760148,  0.22061206, -0.23743176,  0.20440523,
       -0.05492518,  0.08233099,  0.14771615, -0.3847315 ,  0.35306096,
        0.06957682,  0.25351733,  0.25282878,  0.09940425,  0.10070784,
        0.2629964 , -0.168038  ,  0.12743732, -0.2336523 , -0.26059604,
       -0.08014607,  0.22298959, -0.00285875, -0.11516835, -0.07916622,
        0.24770644, -0.20618638, -0.5045272 , -0.2963034 , -0.4026901 ,
        0.21374956, -0.06301931,  0.11857571,  0.19401436, -0.03305751,
        0.51753765, -0.12518515, -0.07292145,  0.5125408 ,  0.32581112,
       -0.09140911,  0.10136102, -0.10169743, -0.16989364, -0.18277352,
        0.2366689 ,  0.2156801 , -0.14649333, -0.14269456,  0.12841523],
      dtype=float32)

In [8]:
# check item factors
item_factors[0,:]

array([-0.2520061 , -0.12801759, -0.01807678, -0.06055114, -0.04010666,
        0.04099023,  0.0625717 ,  0.00891204, -0.13964826,  0.08402346,
        0.0650855 ,  0.13047956,  0.03227995,  0.03988666, -0.07124443,
       -0.06424009,  0.02801364, -0.00741817, -0.09139743,  0.02618102,
       -0.01038758,  0.00207332, -0.13542707, -0.09922392, -0.02690951,
        0.11702102,  0.03746996, -0.1585136 , -0.08835025, -0.13124834,
        0.02231358,  0.05603463, -0.0014683 ,  0.0878399 ,  0.04198717,
        0.19678356, -0.13217868, -0.15097322,  0.16004062, -0.01319988,
       -0.02964439,  0.06364242, -0.0199494 ,  0.00027959,  0.01765113,
        0.14233795,  0.11001149, -0.06930998, -0.03381407, -0.05987406],
      dtype=float32)

# Utility Functions

# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [9]:
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,1,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A39QHP5WLON5HV,5.0,There are usually one or more of my cats watch...,2013-09-14,usually cat watch tv stay trouble dvd play lik...
1,104,B00005MF9V,LitterMaid Universal Cat Privacy Tent (LMT100),"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A366V0GCEPH5CX,5.0,My cats love it and so do I. I no longer have ...,2013-02-02,cat love longer cat litter fly floor litter fl...
2,133,B00005MF9T,LitterMaid LM500 Automated Litter Box,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",ALWWS8QBYN80B,1.0,I have one female cat that weighs under 10 pou...,2004-11-17,female cat weigh pound year old use everclean ...
3,153,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A3PVI3NE7OY1SP,5.0,I love these. They make the clean up so much e...,2013-09-26,love clean easy clean box manually use issue w...
4,154,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A2H83XMHUVDLJY,4.0,"I love this litter box. I do not use the lids,...",2014-06-26,love litter box use lid use receptacle tear cr...
41564,111601,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],AV34KNYW82YSS,4.0,Pulled lots of hair out of my Labs coat. Didn'...,2014-07-18,pulled lot hair labs coat think prove wrong co...
41565,111603,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1YMNTFLNDYQ1F,5.0,I have been trying to find a rubber bristle br...,2014-07-16,try rubber bristle brush persian year lose glo...
41566,111604,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1FQ3HRVXA4A5B,5.0,Great product to use on your pets knowing this...,2014-07-11,great product use pet know gentle rubber damag...
41567,111605,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A3OP6CI0XCRQXO,5.0,I bought a second one because I have two cats ...,2014-07-22,buy second cat american short hair buy brush m...
41568,111606,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A11LC938XF35XN,5.0,Our dogs love getting brushed with this. It m...,2014-07-17,dog love brush massage remove heavy undercoat ...


In [10]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [11]:
print(test_user_history)

                  reviewerID                                  asin
0      A04173782GDZSQ91AJ7OD              [B0090Z9AYS, B00CPDWT2M]
1      A042274212BJJVOBS4Q85              [B005AZ4M3Q, B00771WQIY]
2       A0436342QLT4257JODYJ  [B0018CDR68, B003SJTM8Q, B00474A3DY]
3      A04795073FIBKY8GSLZYI              [B001PKT30M, B005DGI2RY]
4      A06658082A27F4VB5UG8E              [B000TZ1TTM, B0019VUHH0]
...                      ...                                   ...
18993          AZYJE40XW6MFG              [B00HVAKJZS, B00IDZT294]
18994          AZZ56WF4X19G2                          [B004A7X218]
18995          AZZNK89PXD006  [B0002DHV16, B005BP8MQ8, B009RTX4SU]
18996          AZZV9PDNMCOZW              [B007EQL390, B00ISBWVT6]
18997          AZZYW4YOE1B6E  [B0002AQPA2, B0002AQPA2, B0002ARQV4]

[18998 rows x 2 columns]


## Preparing Dataset for Surprise's Algorithm

In [12]:
# create reader
reader = Reader(rating_scale=(1,5))
# generate data required for surprise
data = Dataset.load_from_df(train[["reviewerID", "asin", "overall"]], reader)
# generating trainset
trainset = data.build_full_trainset()

## Instantiate Pre-Initialised Matrix Factorization (Paragraph Vector)

In [13]:
# instantiating mod-ecf
mod_ecf = cf.PreInitialisedMF(user_map=user_idx_map,
                              item_map=item_idx_map,
                              user_factor=user_factors,
                              item_factor=item_factors,
                              learning_rate=LR_ALL,
                              beta=BETA,
                              num_epochs=N_EPOCHS,
                              num_factors=50)

In [14]:
%%time
# fitting to training data
mod_ecf.fit(trainset, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
CPU times: user 8min 46s, sys: 2.12 s, total: 8min 48s
Wall time: 8min 53s


In [15]:
%%time
# generate candidate items for user to predict rating
testset = trainset.build_anti_testset()

CPU times: user 51.7 s, sys: 2.64 s, total: 54.4 s
Wall time: 55.1 s


In [16]:
%%time
# predict ratings for all pairs (u, i) that are NOT in the training set
candidate_items = mod_ecf.test(testset, verbose=False)

CPU times: user 12min 31s, sys: 6min 13s, total: 18min 45s
Wall time: 21min 55s


## Loop through N = {10, 25, 30, 45}

In [17]:
# generate item popularity
item_popularity = evaluate_model.generate_item_popularity(train)

In [18]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    # top_ns = get_top_n(candidate_items, n)
    top_ns = mod_ecf.get_top_n(candidate_items, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name = 'MOD-ECF',
                                                    top_ns = top_ns,
                                                    user_rating_history = test_user_history, 
                                                    item_popularity = item_popularity, 
                                                    n = n,
                                                    mf_based = True))
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

The MOD-ECF has an average recall@10: 0.00485, average novelty@10: 0.95198
The MOD-ECF has an average recall@25: 0.01231, average novelty@25: 0.95818
The MOD-ECF has an average recall@30: 0.01462, average novelty@30: 0.95962
The MOD-ECF has an average recall@45: 0.02073, average novelty@45: 0.96280


# Evaluate N-Recommendations

## N=10

In [19]:
top_ns_10 = n_recommendations[10][0]
utilities.retrieve_recommendations(train, top_ns_10, mf_based=True)

For user: A3T87QAUUPTMZK:
Purchase History:
             asin                                              title
43366  B0010OSIHW  Zoo Med Eco Earth Compressed Coconut Fiber Sub...
45100  B00167VVP4   Zoo Med Eco Earth Loose Coconut Fiber Substra...
46998  B0019IJXD2       Zoo Med Reptile Fogger Terrarium Humidifier 

Recommending:

         asin                                              title
0  B0002AS1CC                 Bergan Turbo Scratcher Accessories
1  B00006IX59                         Chuckit! Dog Ball Launcher
2  B003FM7JZY                        Timothy Gold Hay, 5Lb, Blue
3  B00025K1GG                  Hikari Freeze Dried Tubifex Worms
4  B000255NCI                               API Master Test Kits
5  B000AAIAJS                             Vitakraft Rabbit Slims
6  B0002DJONY                            Vittles Vault Stackable
7  B000084F4T  Purina Pro Plan Focus Weight Management Chicke...
8  B000255OIG       Stewart Freeze Dried Treats 14 oz Beef Liver
9  B0002563O0 

## N=25

In [20]:
top_ns_25 = n_recommendations[25][0]
utilities.retrieve_recommendations(train, top_ns_25, mf_based=True)

For user: A1XEZIHQIUAOR1:
Purchase History:
             asin                                              title
1487   B000084E6V                      Nylabone Dental Dinosaur Chew
1727   B000084E6V                      Nylabone Dental Dinosaur Chew
28789  B0006VMN4O  Pioneer Pet SmartCat Peek-A-Prize Toy Box with...
37658  B000JZ1WSU                     SmartCat 3836 Tick Tock Teaser
42603  B000XZDV44  Hill'S Science Diet Kitten Savory Salmon Entre...
43867  B0012KCUOG  Whisker Lickin'S Soft &amp; Delicious Chicken ...

Recommending:

          asin                                              title
0   B00006IX5A               Canine Hardware Chuckit! Tennis Ball
1   B000084DXS  Hill's Science Diet Puppy Chicken &amp; Barley...
2   B000084E7Y                  Multipet Duckworth Duck Large 13"
3   B000084F4T  Purina Pro Plan Focus Weight Management Chicke...
4   B000084F80    Hartz Chew 'n Clean Bacon Flavored Dog Chew Toy
5   B00008434T                Ticked Off Pets Tick Remover, W

## N=30

In [21]:
top_ns_30 = n_recommendations[30][0]
utilities.retrieve_recommendations(train, top_ns_30, mf_based=True)

For user: A89LQAXW1IY6S:
Purchase History:
             asin                                              title
12819  B0002ARP2O  Marshall Ferret Deluxe Leisure Lounge, Pattern...
12825  B0002ARP2O  Marshall Ferret Deluxe Leisure Lounge, Pattern...
38837  B000MD3NLS  MidWest Homes for Pets Snap'y Fit Stainless St...
42683  B000Y8UNAU  Pro Select Fleece Cat Perch Covers - Comfortab...
46465  B0018CJZ32                         SmartCat Corner Litter Box

Recommending:

          asin                                              title
0   B000084DXS  Hill's Science Diet Puppy Chicken &amp; Barley...
1   B000084F4T  Purina Pro Plan Focus Weight Management Chicke...
2   B00008434T                Ticked Off Pets Tick Remover, White
3   B0000AH3UK            Merrick Flossies Tendon Chews Pack of 5
4   B00025K1GQ  Hikari Bio-Pure Freeze Dried Blood Worms for P...
5   B00025YVG2                                            Wardley
6   B00025Z6FW  Hikari Cichlid Gold Floating Baby Pellets for ...

## N=45

In [22]:
top_ns_45 = n_recommendations[45][0]
utilities.retrieve_recommendations(train, top_ns_45, mf_based=True)

For user: A3C2ECIXEQ0YFQ:
Purchase History:
             asin                                              title
30751  B0009YS4P0  Nutri-Vet Hip &amp; Joint Extra Strength Chewa...
37067  B000IBRI2Y              Dog Dazer II Ultrasonic Dog Deterrent
40175  B000OV4VAU                        Nutri-Vet Alaska Salmon Oil
53623  B0029NQTI8  Pedigree Choice Cuts Variety Pack Lamb/Vegetab...

Recommending:

          asin                                              title
0   B00025K1GQ  Hikari Bio-Pure Freeze Dried Blood Worms for P...
1   B003R0MFX0  Tiki Cat Napili Wild Salmon And Chicken In Chi...
2   B000084F4T  Purina Pro Plan Focus Weight Management Chicke...
3   B002BR9GUU  Acana Wild Prairie Grain-Free Dry Dog Food, 29...
4   B00028ZLD6                     Panacur Canine Dewormer 1 gram
5   B0012KB4U2  Purina Whisker Lickin's Crunchy &amp; Yummy Ch...
6   B001G0NKVO  Purina Friskies Indoor Adult Wet Cat Food - (2...
7   B00025Z6Q6  Tetra TetraCichlid Balanced Diet Flakes Food f...
8